In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
def getdata(path):
    import pandas as pd
    import numpy as np
    
    coltypes = {'Pclass':'category', 'Sex':'category', 'Cabin':'str'}

    df = pd.read_csv(path, dtype = coltypes)
    df['Embarked'] = pd.Categorical(df.Embarked, categories=['C','Q','S']).fillna('S')
    df['Age'] = df.Age.fillna(df.Age.mean())
    df['Fare'] = df.Fare.fillna(df.Fare.mean())
    df['Cabin'] = df.Cabin.fillna('Unknown')
    df['CabinChar'] = pd.Categorical(df.Cabin.str[0])
    df['Title'] = pd.Categorical(df.Name.str.split('[,.]+').str[1])
    
    return(df)

df = getdata('../input/train.csv')

print(df.head(1).transpose())
print(df.info())

In [ ]:
response = 'Survived'
regressors_cat = ["Pclass", "Sex", "Embarked", "CabinChar", "Title"]
regressors_num = ["Age", "SibSp", "Parch", "Fare"]
X = df[regressors_cat + regressors_num]
y = df[response]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=9527, stratify=y)
print('After train_test_split, shapes =', X_train.shape, X_test.shape, y_train.shape, y_test.shape)

from sklearn.preprocessing import OneHotEncoder, StandardScaler
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_cat = ohe.fit_transform(X_train[regressors_cat])
X_test_cat = ohe.transform(X_test[regressors_cat])

stdscaler = StandardScaler()
X_train_num = stdscaler.fit_transform(X_train[regressors_num])
X_test_num = stdscaler.transform(X_test[regressors_num])

X_train_process = np.concatenate([X_train_cat, X_train_num], axis=1)
X_test_process = np.concatenate([X_test_cat, X_test_num], axis=1)
print('After onehot.fit_transform, shapes =', X_train_process.shape, X_test_process.shape, y_train.shape, y_test.shape)

In [ ]:
from xgboost import XGBClassifier
modelxgb = XGBClassifier(n_jobs=4, n_estimator=1000, max_depth=10, learning_rate=0.05, reg_lambda=0.1)
modelxgb.fit(X_train_process, y_train
             , eval_set=[(X_train_process, y_train), (X_test_process, y_test)], eval_metric=['error',"logloss"])

from sklearn.metrics import accuracy_score
y_pred = modelxgb.predict(X_test_process)
print('Accuracy of test data:', accuracy_score(y_test, y_pred))


In [ ]:
df_submit = getdata('../input/test.csv')
X_submit = df_submit[regressors_cat + regressors_num]
X_submit_cat = ohe.transform(X_submit[regressors_cat])
X_submit_num = stdscaler.transform(X_submit[regressors_num])
X_submit_process = np.concatenate([X_submit_cat, X_submit_num], axis=1)

df_submit_final = pd.DataFrame(df_submit['PassengerId'])
df_submit_final['Survived'] = pd.Series(modelxgb.predict(X_submit_process))

In [ ]:
df_submit_final.to_csv('output.csv', index=False)